#### Universidad Autónoma de Aguascalientes
#### Departamento: Ciencias de la Computación
#### Carrera: Ingenieria en Computación Inteligente
#### Curso: Machine Learning y Deep Learning 
#### Maestro: Dr. Francisco Javier Luna Rosas

#### Alumno: Jorge Hernández
#### Semestre: Enero - Junio del 2025

In [1]:
# LIBRERIAS
import cv2
import math
import mediapipe as mp
import numpy as np
import pandas as pd
import os
import mediapipe as mp

In [2]:
Emociones = ["Alegria","Enojado","Tristeza","Sorprendido","Neutral"]
direccion = "../Examen_Analisis_de_Sentimientos_Face_Detection/Emociones"
cantidad_frame = 350

In [3]:
for index in range(len(Emociones)):
    try:
        os.mkdir(direccion+"/"+Emociones[index])
    except FileExistsError:
        print(f"❌ La Carpeta {Emociones[index]} Ya Existe") 


In [4]:
values = []

In [5]:
def calcular_distancias(punto1, punto2):
    x1, y1, z1 = punto1.x, punto1.y, punto1.z
    x2, y2, z2 = punto2.x, punto2.y, punto2.z

    distancia = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
    return distancia

In [6]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True)

In [ ]:
def capturar_rostro(emocion,cantidad_frame,persona):

    cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
    faceClassif = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    count = 0

    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    ref_x, ref_y, ref_w, ref_h = frame_width // 4, frame_height // 4, frame_width // 2, frame_height // 2

    
    while True:
        ret, frame = cap.read()

        if ret == False:
            print("❌ ERROR EN LA CAPTURA DE IMAGEN") 
            break

        results = face_mesh.process(frame)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        auxFrame = frame.copy()


        faces = faceClassif.detectMultiScale(gray,
            scaleFactor=1.1,
            minNeighbors=20,
            minSize=(30,30),
            maxSize=(500,500))
        
        cv2.rectangle(frame, (ref_x, ref_y), (ref_x + ref_w, ref_y + ref_h), (0, 0, 255), 2)  # Rojo si el rostro no está dentro


        for (x,y,w,h) in faces:
            
            faces = sorted(faces, key=lambda x: x[2] * x[3], reverse=True)
            x, y, w, h = faces[0]

            cv2.rectangle(frame, (x,y), (x+w,y+h),(0,255,0),2)
            cara = auxFrame[y:y+h,x:x+w]
            cara = cv2.resize(cara,(150,150), interpolation=cv2.INTER_CUBIC)

            cv2.putText(frame, f'Emocion: {emocion}: {count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            if x >= ref_x and y >= ref_y and (x + w) <= (ref_x + ref_w) and (y + h) <= (ref_y + ref_h):

                count += 1
                cv2.imwrite(direccion+"/"+emocion+"/"+emocion+"_{}_{}.jpg".format(count,persona),cara)
                
                if results.multi_face_landmarks:
                    for face_landmarks in results.multi_face_landmarks:
                        connection_drawing_spec = mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=1)
                        landmark_drawing_spec = mp_drawing.DrawingSpec(color=(0, 0, 0), thickness=0, circle_radius=0)

                        mp_drawing.draw_landmarks(frame, face_landmarks,
                                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                                    landmark_drawing_spec=landmark_drawing_spec,
                                    connection_drawing_spec=connection_drawing_spec)

                        # PUNTOS DE BOCA
                        # Punto medio labio superior
                        p13 = face_landmarks.landmark[13]
                        # Punto medio labio inferior
                        p14 = face_landmarks.landmark[14]

                        # Punto externo izquierdo
                        p61 = face_landmarks.landmark[61]
                        # Punto extremo derecho
                        p306 = face_landmarks.landmark[306]

                        # PUNTOS DE CEJA
                        # CEJA IZQUIERDA
                        p65 = face_landmarks.landmark[65]
                        # CEJA DERECHA
                        p295 = face_landmarks.landmark[295]

                        # PUNTOS DE OJO
                        # OJO IZQUIERDO
                        # PUNTO SUPERIOR
                        p159 = face_landmarks.landmark[159]
                        # PUNTO INFERIOR
                        p145 = face_landmarks.landmark[145]

                        # OJO DERECHO
                        # PUNTO SUPERIOR
                        p386 = face_landmarks.landmark[386]
                        # PUNTO INFERIOR
                        p374 = face_landmarks.landmark[374]
                        
                        Dist_Ojo_Izq = calcular_distancias(p65,p159)
                        Dist_Ojo_Der = calcular_distancias(p295,p386)

                        Aper_Ojo_Der = calcular_distancias(p386,p374)
                        Aper_Ojo_Izq = calcular_distancias(p159,p145)

                        Ancho_Boca = calcular_distancias(p61,p306)
                        Apertura_Boca = calcular_distancias(p13,p14)

                        values.append([persona,Dist_Ojo_Izq,Dist_Ojo_Der,Aper_Ojo_Der,Aper_Ojo_Izq,Ancho_Boca,Apertura_Boca,emocion])

            else:
                cv2.putText(frame, "Posiciona tu rostro en el cuadro", (ref_x, ref_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        cv2.imshow("Capturando...", frame)
        
        if cv2.waitKey(1) & 0xFF == 27 or count>= cantidad_frame:  # Presiona 'Esc' para salir
            break

    cap.release()
    cv2.destroyAllWindows()

In [8]:
#Alegria
capturar_rostro(Emociones[0],cantidad_frame,"0")

In [9]:
#Enojado
capturar_rostro(Emociones[1],cantidad_frame,"0")

In [10]:
#Tristeza
capturar_rostro(Emociones[2],cantidad_frame,"0")

In [11]:
#Sorprendido
capturar_rostro(Emociones[3],cantidad_frame,"0")

In [12]:
#Neutral
capturar_rostro(Emociones[4],cantidad_frame,"0")

In [ ]:
df = pd.DataFrame(values, columns= ["Sujeto","Distancia Entre Ceja y Ojo Izquierdo","Distancia Entre Ceja y Ojo Derecho","Apertura de Ojo Derecho","Apertura de Ojo Izquierdo","Ancho de Boca","Apertura de Boca","Expresion"])
df.to_csv("data.csv", index=False)
print("Archivo Generado.")

Archivo Generado.
